In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import boto3

In [14]:
s3 = boto3.client('s3')

bucket = 'datascience-datasetsbucket'
prefix = 'rdj-airbnb-prices/'
obj_list = s3.list_objects_v2(Bucket=bucket,Prefix=prefix)
local_dir = '../data/'

keys = [obj['Key'] for obj in obj_list['Contents']]

for key in [key for key in keys if key not in [prefix,prefix + 'total_data.csv']]:
    local_file_path = key.replace(prefix,local_dir)
    s3.download_file(bucket, key, local_file_path)

In [ ]:
dfs = []
for filename in os.listdir('../data'):
    if filename in ['test.csv','train.csv']:
        pass
    else:
        df = pd.read_csv(os.path.join('../data/',filename),dtype={'id':int})
        df = df.apply(lambda x: x.str.strip() if isinstance(x, str) else x)
        dfs.append(df)
df = pd.concat(dfs)

In [4]:
#as there are a lot of columns for simplicity I will just focus on categorical and numerical data, will also remove location data that is not needed (ie country/city/state).
df = df[['id','last_scraped','host_since',
       'host_location', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'zipcode', 'market',
       'smart_location', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', 'square_feet',
       'price', 'weekly_price', 'monthly_price', 'security_deposit',
       'cleaning_fee', 'guests_included', 'extra_people', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
       'availability_30','availability_60', 'availability_90', 'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'license',
       'jurisdiction_names', 'instant_bookable', 'is_business_travel_ready',
       'cancellation_policy', 'require_guest_profile_picture',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']]


In [6]:
## split dataset
train, test = train_test_split(df, test_size=0.2)

train.to_csv('../data/train.csv',index=False)
test.to_csv('../data/test.csv', index=False)